In [199]:
"""
Date: Dec 2, 2022
Author: Sashka Warner
Desc: Load site data and find nearest sites to a given location based on travel type (eg walking, biking, etc)
Status: Working on filtering out nearby points to a given user to try to conserve number of calls to Google Maps API distance matrix

Overall goal ->
Recommendation tool: Create a web tool that 
recommends the most accessible service providers to a 
client based on a preferred mode of transportation 
(e.g. bike, walk, bus, car) and services needed.

Suggested idea : Create a web app (e.g. Flask app with 
Python and Google Maps API) that based on a client’s address,
services needed and preferred mode of transportation could 
recommend in a map the top N most accessible service providers 
for each need.  
"""

import os
import requests
import json
import folium
from folium.plugins import MarkerCluster
import pandas as pd
from geopandas import GeoDataFrame
import googlemaps
from itertools import tee
from dotenv import load_dotenv
from shapely.geometry import Point
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load environment variables
load_dotenv()

# Get API key
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

# Authenticate with Google Maps API 
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

In [95]:
# Load the site data
sites_raw = pd.read_csv("../../uwwi_datasets/uwwi_dataset_sites.csv")

# Inspect the data
sites_raw.head()

,AgencySystem_Name,Agency_Id,Site_Id,Site_AgencyId,Site_CreateStamp,Site_EditStamp,Site_AuditStamp,Site_Status,SiteSystem_Name,SiteSystem_Description,...,SiteAddressus_SiteAddressus_validated,SiteAddressus_SiteAddressus_custom_location,SiteAddressus_SiteAddressus_zip_latitude,SiteAddressus_SiteAddressus_zip_longitude,SiteAddressus_SiteAddressus,SiteOption_PermanentlyInactiveSite,SiteOption_RecordOwner,SiteOption_Accessibility,SiteCustom_NonStandardHoursText,SiteHoursofoperation_ModuleHoursofoperation.open
0,INTEGRATED COMMUNITY SOLUTIONS,1,414.0,1.0,2017-03-16T10:32:21.216012-05:00,2022-05-12T12:53:25.967449-05:00,NaN,active,INTEGRATED COMMUNITY SOLUTIONS,NaN,...,True,False,44.489906,-88.06991,"2605 S ONEIDA ST Suite 106 GREEN BAY WI, 54304",[],['BCUW'],[],Monday-Friday 8am-4:30pm,"[{'day': -1, 'end_min': None, 'end_hour': None..."
1,INTEGRATED COMMUNITY SOLUTIONS,1,418.0,1.0,2017-03-16T10:32:22.867221-05:00,2021-10-21T19:48:22.74763-05:00,NaN,active,ZZZINACTIVE LEAVING HOMELESSNESS BEHIND,NaN,...,False,False,44.489906,-88.06991,2605 South Oneida Street Suite 106 Green Bay W...,[],['BCUW'],[],"Monday-Friday, 8:00am-4:30pm","[{'day': -1, 'end_min': None, 'end_hour': None..."
2,INTERIM HEALTH CARE,2,419.0,2.0,2017-03-16T10:32:23.227182-05:00,2021-11-16T10:53:27.616276-06:00,NaN,active,INTERIM HEALTH CARE,NaN,...,False,False,44.542973,-88.05582,"1600 Shawano Avenue Suite 201 Green Bay WI, 54303",[],['BCUW'],[],Monday-Friday 8:30am-5:00pm,"[{'day': -1, 'end_min': None, 'end_hour': None..."
3,zzinactive_INTERNATIONAL TRANSLATORS,3,420.0,3.0,2017-03-16T10:32:23.567045-05:00,2021-07-27T23:57:41.974815-05:00,NaN,active,zzinactive_VILLA REAL DBA INTERNATIONAL TRANSL...,NaN,...,False,False,44.483376,-88.02269,529 South Jefferson Street Suite 203 Green Bay...,[],['BCUW'],"['Elevators', 'Outside Ramps']","Monday-Friday, 8:00am-5:00pm; Interpreters ava...","[{'day': -1, 'end_min': None, 'end_hour': None..."
4,JACKIE NITSCHKE CENTER,4,421.0,4.0,2017-03-16T10:32:23.869975-05:00,2022-09-28T10:20:58.151543-05:00,NaN,active,JACKIE NITSCHKE CENTER,NaN,...,True,False,44.483376,-88.02269,"630 CHERRY STREET GREEN BAY WI, 54301",[],['BCUW'],[],Business hours: Monday-Thursday 7am-6:30pm; Fr...,"[{'day': -1, 'end_min': None, 'end_hour': None..."


In [118]:
# Make a copy of the data
sites = sites_raw.copy()

# Extract columns of interest
sites = sites[[
    "AgencySystem_Name", 
    "SiteSystem_Name", 
    "Agency_Id", 
    "Site_Id",
    "Site_AgencyId", 
    "Site_Status", 
    "SiteAddressus_SiteAddressus_latitude", 
    "SiteAddressus_SiteAddressus_longitude", 
    "SiteAddressus_SiteAddressus_zip_latitude", 
    "SiteAddressus_SiteAddressus_zip_longitude", 
    "SiteAddressus_SiteAddressus",
    ]]

# Rename location columns
sites.rename(columns = {
    "SiteAddressus_SiteAddressus_latitude": "site_lat",
    "SiteAddressus_SiteAddressus_longitude": "site_long",
    "SiteAddressus_SiteAddressus_zip_latitude": "site_zip_lat",
    "SiteAddressus_SiteAddressus_zip_longitude": "site_zip_long",
    "SiteAddressus_SiteAddressus": "site_address",
}, inplace=True)
#sites.head()

# Check if lat/long data has nulls
#any(sites["site_lat"].isna())
#any(sites["site_long"].isna())

# Remove null lat/long records
sites.dropna(subset=["site_lat", "site_long"], inplace=True)

# Remove sites that have zero values for lat/long TODO: Go back and get zip long/lat if exists?
sites = sites[(sites["site_lat"] != 0) & (sites["site_long"] != 0)]

# Reset index after removing null and zero values
sites.reset_index(drop=True, inplace=True)

# Make sure no nulls exist
#any(sites["site_lat"].isna())
#any(sites["site_long"].isna())
#len(sites[sites["site_lat"] == 0])
#len(sites[sites["site_long"] == 0])

# Check that all of the lat/long data is of numeric type
#all(sites["site_lat"].apply(lambda x: isinstance(x, float)))
#ll(sites["site_long"].apply(lambda x: isinstance(x, float)))

# Check if any of the Site IDs have nulls
#any(sites["Site_Id"].isna())


In [111]:
# Inspect the sites data
print(sites.shape)

(19438, 11)


In [188]:
# Specify start address for testing
# Google maps takes coordinates in <Lat, Long> format
# E.G. (42.939304, -89.568820)
start_lat = 42.939304
start_long = -89.568820

# Create test gdf so we can project the point
start_point = Point(start_long, start_lat)
start_df = pd.DataFrame({"user": [1], "lat": [start_lat]})
start_gdf = GeoDataFrame(start_df, crs="EPSG:4326", geometry=[start_point]).to_crs("EPSG:3070")
print(start_gdf)

# Get one of the site addresses for example
dest_lat = sites.loc[0, "site_lat"]
dest_long = sites.loc[0, "site_long"]
dest_point = (dest_lat, dest_long)

   user        lat                       geometry
0     1  42.939304  POINT (555179.369 274164.820)


In [191]:
# Convert site dataframe to geodataframe
geometry = [Point(xy) for xy in zip(sites.site_long, sites.site_lat)]

#df = sites.drop(['Lon', 'Lat'], axis=1)
sites_gdf = GeoDataFrame(sites, crs="EPSG:4326", geometry=geometry)
#sites_gdf.head()

In [192]:
# Reproject data to Wisconsin coordinate system
# See: https://efotg.sc.egov.usda.gov/references/public/WI/coordinate_system_comparison.pdf
# TODO: Try to increase accuracy by seeing if way to project data on the fly using the 
# Wisconsin Coordinate Reference System for a given county?
# https://epsg.io/3070
sites_gdf = sites_gdf.to_crs("EPSG:3070") #Units: meters

In [ ]:
# Load wisconsin data for plotting (may take a second...)
#wisconsin_geojson = "https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wi_wisconsin_zip_codes_geo.min.json"
#wisconsin = json.loads(requests.get(wisconsin_geojson).text)

In [122]:
# Creating the map centered at Wisconsin state
m = folium.Map(location=[44.808444, -89.673194], 
               tiles="cartodbpositron", 
               zoom_start=6.8)

#Create feature group so we can toggle data visibility
fg = folium.FeatureGroup(name="Sites", show=False)
m.add_child(fg)
marker_cluster = MarkerCluster().add_to(fg)

# Create a geometry list from the GeoDataFrame
#site_points = [[point.xy[1][0], point.xy[0][0]] for point in sites_gdf.loc[0:100,"geometry"]] # Note: Doesn't work with projected CRS (Folium leaflet need EPSG:4326)

# Note that Folium wants data in <Lat, Long> format
n = sites_gdf.shape[0]
for i in range(n-1):
    coords = (sites_gdf.loc[i, "site_lat"], sites_gdf.loc[i, "site_long"])
    folium.Marker(
        location= coords,
        popup="Coordinates: " + str(coords),
        control=True,
    ).add_to(marker_cluster)

folium.LayerControl().add_to(m)

In [ ]:
# Display map
# NOTE: Sites include points from all over the US
#m #maps can take a long time to load"""

In [196]:
# Create buffer from start point
# FIXME: Fix this to get proper buffer and extract intersecting points -> probably just a lat/long/crs mixup
WALKING_MAX_RADIUS_M = 100000
search_buffer = start_gdf.buffer(WALKING_MAX_RADIUS_M)
print(search_buffer)
#search_buffer.crs

# Get sites within the buffer
any(sites_gdf.within(search_buffer))
#sites_gdf[sites_gdf.within(search_buffer)]
#sites_gdf.crs

0    POLYGON ((655179.369 274164.820, 654697.842 26...
dtype: geometry


/Users/sashkawarner/repos/OLD_UWWi_DataDive_Dec22/venv/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


False

In [197]:
# Get distance by walking from address to destination
# CAUTION: Requests to the distance api cost $ / count towards rate limits so try to conserve amt. of requests
# Current idea: first remove locations that are definitely outside
# the service area 
# EG create buffer around point 
# (10 mi for walking, 25 mi for cycling, and 50-75 mi for train and driving?)
# And select sites within that buffer
# Then within that buffer, calculate distances by a given mode
# of transportation and find the closest distances
"""
distance = gmaps.distance_matrix(
    origins=start_point,
    destinations=dest_point, 
    mode="walking", 
    units="imperial")
    """

'\ndistance = gmaps.distance_matrix(\n    origins=start_point,\n    destinations=dest_point, \n    mode="walking", \n    units="imperial")\n    '

In [198]:
#print(distance)
"""
Sample response from distance matrix call:
{'destination_addresses': ['2605 S Oneida St, Green Bay, WI 54304, USA'],
 'origin_addresses': ['1510 Speedway Rd, Verona, WI 53593, USA'],
 'rows': [{'elements': [{'distance': {'text': '151 mi', 'value': 242715},
     'duration': {'text': '2 days 1 hour', 'value': 177155},
     'status': 'OK'}]}],
 'status': 'OK'}
"""


"\nSample response from distance matrix call:\n{'destination_addresses': ['2605 S Oneida St, Green Bay, WI 54304, USA'],\n 'origin_addresses': ['1510 Speedway Rd, Verona, WI 53593, USA'],\n 'rows': [{'elements': [{'distance': {'text': '151 mi', 'value': 242715},\n     'duration': {'text': '2 days 1 hour', 'value': 177155},\n     'status': 'OK'}]}],\n 'status': 'OK'}\n"